In [2]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import dates as mpl_dates
from datetime import datetime, timedelta
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pycaret

In [3]:
from pycaret.regression import *

In [4]:
df1 = pd.read_csv('assets/merged_data_teledyne_and_ENE00960_R.csv', parse_dates=["DataDate"])
df1

,DataDate,PM2.5_Ref,Hum,Temp,PM2_5
0,01/10/2023 17:54,7.276371,100.5,26.80,4.0
1,01/10/2023 17:55,7.128142,100.5,26.79,4.0
2,01/10/2023 17:56,7.118874,100.5,26.79,4.0
3,01/10/2023 17:57,7.077153,100.5,26.79,2.0
4,01/10/2023 17:58,7.055617,100.5,26.77,4.0
...,...,...,...,...,...
90207,22/02/2024 9:42,27.340340,91.5,31.33,14.0
90208,22/02/2024 9:43,27.341790,91.1,31.35,15.0
90209,22/02/2024 9:44,27.313970,90.9,31.41,17.0
90210,22/02/2024 9:45,27.376200,90.8,31.43,17.0


In [5]:
df1.isnull().sum()

DataDate     0
PM2.5_Ref    0
Hum          0
Temp         0
PM2_5        0
dtype: int64

In [6]:
filtered_df1 = df1[["PM2.5_Ref","Hum", "Temp", "PM2_5"]]
filtered_df1
# df1 = df1[["PM2.5_Ref", "PM2_5_LCS"]]

,PM2.5_Ref,Hum,Temp,PM2_5
0,7.276371,100.5,26.80,4.0
1,7.128142,100.5,26.79,4.0
2,7.118874,100.5,26.79,4.0
3,7.077153,100.5,26.79,2.0
4,7.055617,100.5,26.77,4.0
...,...,...,...,...
90207,27.340340,91.5,31.33,14.0
90208,27.341790,91.1,31.35,15.0
90209,27.313970,90.9,31.41,17.0
90210,27.376200,90.8,31.43,17.0


In [7]:
s = setup(filtered_df1, target = 'PM2.5_Ref', session_id = 123)

,Description,Value
0,Session id,123
1,Target,PM2.5_Ref
2,Target type,Regression
3,Original data shape,"(90212, 4)"
4,Transformed data shape,"(90212, 4)"
5,Transformed train set shape,"(63148, 4)"
6,Transformed test set shape,"(27064, 4)"
7,Numeric features,3
8,Preprocess,True
9,Imputation type,simple


In [8]:
# compare baseline models
best1 = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,6.9372,150.9931,12.2856,0.9582,0.3298,0.2721,6.9850
et,Extra Trees Regressor,7.0605,163.5638,12.7864,0.9547,0.3443,0.2786,3.5290
lightgbm,Light Gradient Boosting Machine,8.7111,182.6841,13.5133,0.9494,0.3472,0.3191,0.3330
knn,K Neighbors Regressor,8.2166,206.9519,14.3816,0.9426,0.3459,0.2963,0.4310
dt,Decision Tree Regressor,7.5651,230.8735,15.1864,0.9361,0.3919,0.2961,0.2500
gbr,Gradient Boosting Regressor,10.3345,268.2487,16.3763,0.9256,0.3722,0.3531,1.4280
ada,AdaBoost Regressor,20.9535,657.4333,25.5912,0.8178,0.7752,1.1889,1.3410
lar,Least Angle Regression,16.6286,726.2065,26.9425,0.7988,0.5951,0.5890,0.0520
br,Bayesian Ridge,16.6280,726.2065,26.9425,0.7988,0.5951,0.5890,0.0420
ridge,Ridge Regression,16.6286,726.2065,26.9425,0.7988,0.5951,0.5890,0.0820


In [12]:
# predict on test set
holdout_pred1 = predict_model(best1)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,6.8907,149.4852,12.2264,0.9572,0.3296,0.2698


In [13]:
save_model(best1, 'assets/correction_factor_random_forest_sensor960-25-April-2024')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Hum', 'Temp', 'PM2_5'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('trained_model',
                  RandomForestRegressor(n_jobs=-1, random_state=123))]),
 'assets/correction_factor_random_forest_sensor960-25-April-2024.pkl')

In [15]:
import pickle
import gzip

def compress_pickle_gzip(file_path, data):
    with gzip.open(file_path, 'wb') as f:
        pickle.dump(data, f)

# Usage
model = 'assets/correction_factor_random_forest_sensor960-25-April-2024.pkl'  # This should be your loaded model
file_path = 'assets/model.pkl.gz'  # The .gz extension indicates gzip compression
compress_pickle_gzip(file_path, model)
